In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import polars as pl
from polars.functions import col
from stanza.pipeline.core import Pipeline
from stanza.utils.conll import CoNLL, Document

from label_legends.preprocess import load_data, preprocess
from label_legends.util import CONLL_DIR, ROOT, RESOURCE

RESOURCE

/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PosixPath('/home/lukas/Programming/uni/nlp-ie-label-legends/resource')

# Download files

In [21]:
# ! wget --output-document resource/edos_labelled_individual_annotations.csv https://github.com/rewire-online/edos/raw/refs/heads/main/data/edos_labelled_individual_annotations.csv 

/home/lukas/.local/share/uv/python/cpython-3.12.5-linux-x86_64-gnu/lib/python3.12/pty.py:95: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

  pid, fd = os.forkpty()


--2024-11-26 23:08:43--  https://github.com/rewire-online/edos/raw/refs/heads/main/data/edos_labelled_individual_annotations.csv


Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... 

connected.


HTTP request sent, awaiting response... 

302 Found
Location: https://raw.githubusercontent.com/rewire-online/edos/refs/heads/main/data/edos_labelled_individual_annotations.csv [following]
--2024-11-26 23:08:44--  https://raw.githubusercontent.com/rewire-online/edos/refs/heads/main/data/edos_labelled_individual_annotations.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... 

connected.
HTTP request sent, awaiting response... 





          resource/   0%[                    ]       0  --.-KB/s               






# Load file

In [3]:
df = pl.scan_csv(RESOURCE / "edos_labelled_individual_annotations.csv")
df.collect()

rewire_id,text,annotator,label_sexist,label_category,label_vector,split
str,str,i64,str,str,str,str
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",17,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",2,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",6,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-1""","""What do you guys think about f…",17,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-1""","""What do you guys think about f…",15,"""not sexist""","""none""","""none""","""train"""
…,…,…,…,…,…,…
"""sexism2022_english-9998""","""Since 1973, America has sancti…",2,"""sexist""","""4. prejudiced discussions""","""4.2 supporting systemic discri…","""test"""
"""sexism2022_english-9998""","""Since 1973, America has sancti…",16,"""not sexist""","""none""","""none""","""test"""
"""sexism2022_english-9999""","""Laura Bates Lol she looks like…",2,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""


In [4]:
df.filter( col("split") == "train").collect()

rewire_id,text,annotator,label_sexist,label_category,label_vector,split
str,str,i64,str,str,str,str
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",17,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",2,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",6,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-1""","""What do you guys think about f…",17,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-1""","""What do you guys think about f…",15,"""not sexist""","""none""","""none""","""train"""
…,…,…,…,…,…,…
"""sexism2022_english-9995""","""[USER] *nod* i try to balance …",15,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-9995""","""[USER] *nod* i try to balance …",14,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-9999""","""Laura Bates Lol she looks like…",2,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""


In [5]:
df = load_data()

series = list(df.select(col("text")).collect().to_series())
series

['[USER] I wonder what keeps that witch looking quite young for being 70. Maybe we all should start drinking blood? 😄',
 '[USER] I wonder what keeps that witch looking quite young for being 70. Maybe we all should start drinking blood? 😄',
 '[USER] I wonder what keeps that witch looking quite young for being 70. Maybe we all should start drinking blood? 😄',
 'What do you guys think about female "incels"? I\'m not trying to be argumentative, I\'m just genuinely curious what you guys think of females who feel like shit about themselves and feel they are treated poorly by "chads."',
 'What do you guys think about female "incels"? I\'m not trying to be argumentative, I\'m just genuinely curious what you guys think of females who feel like shit about themselves and feel they are treated poorly by "chads."',
 'What do you guys think about female "incels"? I\'m not trying to be argumentative, I\'m just genuinely curious what you guys think of females who feel like shit about themselves and fe

In [6]:
nlp = Pipeline('en', processors='tokenize,lemma,pos')

2024-11-26 23:49:30 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES



2024-11-26 23:49:30 INFO: Downloaded file to /home/lukas/stanza_resources/resources.json


2024-11-26 23:49:30 WARNING: Language en package default expects mwt, which has been added


2024-11-26 23:49:31 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |



2024-11-26 23:49:31 INFO: Using device: cpu


...

2024-11-26 23:49:33 INFO: Loading: mwt


/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/stanza/models/mwt/trainer.py:201: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoin

/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/stanza/models/pos/trainer.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoin

2024-11-26 23:49:33 INFO: Loading: lemma


/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/stanza/models/lemma/trainer.py:239: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpo

In [9]:
documents_in = [Document([], text=d) for d in series]
# "\n\n".join(series[:100])

docs = nlp(documents_in[:600])
# docs[0]

[
  [
    {
      "id": 1,
      "text": "[",
      "lemma": "[",
      "upos": "PUNCT",
      "xpos": "-LRB-",
      "start_char": 0,
      "end_char": 1,
      "misc": "SpaceAfter=No"
    },
    {
      "id": 2,
      "text": "USER",
      "lemma": "user",
      "upos": "NOUN",
      "xpos": "NN",
      "feats": "Number=Sing",
      "start_char": 1,
      "end_char": 5,
      "misc": "SpaceAfter=No"
    },
    {
      "id": 3,
      "text": "]",
      "lemma": "]",
      "upos": "PUNCT",
      "xpos": "-RRB-",
      "start_char": 5,
      "end_char": 6
    }
  ],
  [
    {
      "id": 1,
      "text": "I",
      "lemma": "I",
      "upos": "PRON",
      "xpos": "PRP",
      "feats": "Case=Nom|Number=Sing|Person=1|PronType=Prs",
      "start_char": 7,
      "end_char": 8
    },
    {
      "id": 2,
      "text": "wonder",
      "lemma": "wonder",
      "upos": "VERB",
      "xpos": "VBP",
      "feats": "Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin",
      "start_char": 9,
  

In [10]:
CONLL_DIR.mkdir(parents=True, exist_ok=True)
for i, doc in enumerate(docs):
    CoNLL.write_doc2conll(doc, str(CONLL_DIR / f"{i}.conll"))
# preprocess(df)